In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
# Load the saved model and tokenizer
loadstr = '/home/ubuntu/tlm-files/tlm/models/tlm-2025-08-05_16-42-11/checkpoint-10500/'
model = AutoModelForMaskedLM.from_pretrained(loadstr)
#tokenizer = BertTokenizerFast.from_pretrained('../coca_tokenized/tokenizer/')
tokenizer = AutoTokenizer.from_pretrained(loadstr)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-11 21:59:15.013195: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-11 21:59:15.032843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757627955.057808   95940 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757627955.065919   95940 cuda_blas.cc:1407] Unable to 

In [2]:
import torch
import torch.nn.functional as F
def l_fill(phrase, fill, model, tokenizer):
    model.eval()

    target_ids = tokenizer.encode(fill, add_special_tokens=False)
    years = list(range(1990, 2020))
    yearly_fill_probs = []
    
    for year in years:
        temporal_phrase = '[YEAR:{}]'.format(year) + phrase
        with torch.no_grad():
            input_ids = tokenizer.encode(temporal_phrase, add_special_tokens=False, return_tensors='pt')
            mask_locs = torch.where(input_ids==tokenizer.mask_token_id)
            num_masks = len(mask_locs[0])
            if num_masks != len(target_ids):
                raise ValueError('Tring to fill {} masks with {} tokens'.format(num_masks, len(target_ids)))

            outputs = model(input_ids=input_ids)
            fill_logits = outputs.logits[mask_locs]
            fill_probs = F.softmax(fill_logits, dim=-1)

            cur_fill_probs = []
            for mask_idx, target_id in enumerate(target_ids):
                cur_fill_probs.append(fill_probs[mask_idx, target_id])
            yearly_fill_probs.append(cur_fill_probs)

    return years, yearly_fill_probs

In [3]:
import torch
import torch.nn.functional as F
def l_fills(phrase, fills, model, tokenizer):
    model.eval()

    fill_target_ids = [tokenizer.encode(fill, add_special_tokens=False) for fill in fills]
    years = list(range(1990, 2020))
    yearly_fill_probs = []
    
    for year in years:
        temporal_phrase = '[YEAR:{}]'.format(year) + phrase
        with torch.no_grad():
            input_ids = tokenizer.encode(temporal_phrase, add_special_tokens=False, return_tensors='pt')
            mask_locs = torch.where(input_ids==tokenizer.mask_token_id)

            outputs = model(input_ids=input_ids)
            fill_logits = outputs.logits[mask_locs]
            fill_sublogits = fill_logits[:, fill_target_ids][0]
            cur_fill_probs = F.softmax(fill_sublogits, dim=0)

            yearly_fill_probs.append(cur_fill_probs)

    return years, yearly_fill_probs

In [4]:
template = 'president [mask] made a speech today'
years, fill_probs = l_fills(template, ['trump', 'obama'], model, tokenizer)

plt.figure()
plt.plot(years, [e[0] for e in fill_probs], label='trump')
plt.plot(years, [e[1] for e in fill_probs], label='obama')
plt.legend()
plt.show()

IndexError: index 0 is out of bounds for dimension 0 with size 0